In [1]:
import vanna
from vanna.remote import VannaDefault
import os
import dotenv

dotenv.load_dotenv()

In [2]:
vanna_model_name = "alckasoc_mymodel" # Your model name from https://vanna.ai/account/profile 
vn = VannaDefault(model=vanna_model_name, api_key=os.getenv("VANNA_API_KEY"))

In [3]:
vn.connect_to_sqlite('data/FPA_FOD_20170508.sqlite')

In [4]:
# Training Data From:
# - https://www.kaggle.com/code/katiej277/sqlite-in-python-intro
# - https://www.kaggle.com/code/preprocessiing/sqlite3-geospatial-analysis-on-wildfire-data
sql_queries = [
"""SELECT SOURCE_REPORTING_UNIT_NAME,count(*) as [count]
FROM fires
GROUP BY SOURCE_REPORTING_UNIT_NAME""",
"""SELECT SOURCE_REPORTING_UNIT_NAME,count(*) as [count]
FROM fires
GROUP BY SOURCE_REPORTING_UNIT_NAME
ORDER BY count
LIMIT 30;""",
""" 
SELECT 
LOCAL_FIRE_REPORT_ID, 
DISCOVERY_DATE AS Date, 
STAT_CAUSE_DESCR AS MainCause, 
FIRE_YEAR AS Year, 
FIRE_SIZE 
FROM fires 
WHERE FIRE_SIZE  > 1000.00 AND year > 2010
ORDER BY FIRE_SIZE 
LIMIT 20;""",
"""SELECT FIRE_SIZE, FIRE_YEAR
FROM fires 
WHERE  FIRE_SIZE > 2000.84
UNION 
SELECT STAT_CAUSE_CODE, FIRE_YEAR
from fires
WHERE FIRE_YEAR > 2010 
LIMIT 40;""",
"""SELECT FIRE_SIZE,
DISCOVERY_TIME,
STAT_CAUSE_CODE, 
SOURCE_REPORTING_UNIT, 
NWCG_REPORTING_UNIT_ID, 
NWCG_REPORTING_UNIT_NAME, 
LEAD(DISCOVERY_TIME, 1, 0) OVER (ORDER BY DISCOVERY_TIME) AS DISCOVERY_TIME,
LEAD(DISCOVERY_TIME, 1, 0) OVER (ORDER BY DISCOVERY_TIME) - DISCOVERY_TIME AS DISCOVERY_TIME_Diff
FROM fires
LIMIT 5;
""",
"""SELECT 
fires.DISCOVERY_TIME AS Time, 
fires.STAT_CAUSE_CODE AS Cause_Code, 
count(distinct FIRE_NAME ) AS FireName,
count(distinct SOURCE_REPORTING_UNIT_NAME ) AS UnitName,
AVG(FIRE_SIZE) AS Fire_Size,  
AVG(Shape) AS Fire_Shape, 
ROUND(AVG(DISCOVERY_TIME)) AS Initial_Date, 
sum(CONT_TIME) AS Time,
sum(COUNTY) AS SubState
FROM fires;""",
"""SELECT 
SUM(CASE 
    WHEN FIRE_YEAR IN (2005)
    THEN FIRE_SIZE
    ELSE 0 END) AS Fire_Id,
SUM(CASE
    WHEN DISCOVERY_TIME IN (1600)
    THEN FIRE_SIZE
    ELSE 0 END) AS  Fire_Size
FROM fires;""",
"""SELECT 
    CASE  
    WHEN FIRE_YEAR > 2010 THEN  'New_Fire_Incedient'
    WHEN FIRE_SIZE > 0.25 THEN 'Biggest_Fire'
    WHEN Shape > 30 AND Shape < 90 THEN 'CirculatingFire'
    END AS Fire_Details, 
    COUNT(DISTINCT FIRE_SIZE) AS Number_Of_Fires
    FROM  fires
    GROUP BY 
        CASE
        WHEN FIRE_YEAR > 2010 THEN 'NewFireIncedient'
        WHEN FIRE_SIZE > 0.25 THEN 'VastFire'
        WHEN Shape > 25 AND Shape < 90 THEN 'CirculatingFire'
    END;""",
"""SELECT DISTINCT NWCG_REPORTING_UNIT_NAME, LOCAL_INCIDENT_ID 
                         FROM fires AS UnitName
                         WHERE EXISTS 
                             (SELECT *
                              FROM fires
                              WHERE FIRE_SIZE > 0.90 
                              AND fires.Fire_name LIKE '%EON%')
                              LIMIT 10;""",
"""SELECT OWNER_DESCR
                         FROM fires 
                         WHERE NWCG_REPORTING_UNIT_ID = 'USCAENF'
                         UNION ALL 
                         SELECT DISTINCT LOCAL_INCIDENT_ID
                         FROM fires
                         WHERE SOURCE_REPORTING_UNIT_NAME = 'Eldorado National Forest'
                         LIMIT 20;""",
]

for sql_query in sql_queries:
    vn.train(sql=sql_query)

Question generated with sql: What is the count of fires for each reporting unit? 
Adding SQL...
Question generated with sql: What are the top 30 reporting units by the number of fires reported? 
Adding SQL...
Question generated with sql: What are the details of the top 20 largest fires in terms of size that occurred after 2010? 
Adding SQL...
Question generated with sql: What are the fire sizes and fire years for fires with a size greater than 2000.84, as well as the cause codes and fire years for fires occurring after 2010, limiting the results to 40 records? 
Adding SQL...
Question generated with sql: What is the difference in discovery time between consecutive fire incidents, along with other details like fire size, cause code, and reporting units? 
Adding SQL...
Question generated with sql: What is the average size and shape of fires, along with the total time and number of fires by cause code, initial date, and substate? 
Adding SQL...
Question generated with sql: What is the tota

In [6]:
vn.ask("Which year had the most fires?")

[{'role': 'system', 'content': 'The user provides a question and you provide SQL. You will only respond with SQL code and not with any explanations.\n\nRespond with only SQL code. Do not answer with any explanations -- just the code.\n\nYou may use the following documentation as a reference for what tables might be available. Use responses to past questions also to guide you:\n\nThis is a SQLite database\n\n'}, {'role': 'user', 'content': 'What are the details of the top 20 largest fires in terms of size that occurred after 2010?'}, {'role': 'assistant', 'content': ' \nSELECT \nLOCAL_FIRE_REPORT_ID, \nDISCOVERY_DATE AS Date, \nSTAT_CAUSE_DESCR AS MainCause, \nFIRE_YEAR AS Year, \nFIRE_SIZE \nFROM fires \nWHERE FIRE_SIZE  > 1000.00 AND year > 2010\nORDER BY FIRE_SIZE \nLIMIT 20;'}, {'role': 'user', 'content': 'What are the fire sizes and fire years for fires with a size greater than 2000.84, as well as the cause codes and fire years for fires occurring after 2010, limiting the results t

: 